<a href="https://colab.research.google.com/github/AhmedAboulezz/Trading/blob/main/Predict_Open_Signal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
#===============================================================================
# CELL 1: Load strategy trades + prices/indicators
#===============================================================================

!pip install pandas numpy scikit-learn --quiet

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")

# لو حابب ترفع الملفات يدوي من جهازك:
# from google.colab import files
# uploaded = files.upload()

# غيّر دول حسب اسماء الملفات عندك:
strategy_csv_path = "StrategyOutputEdited.csv"       # ملف الاستراتيجية
prices_csv_path   = "BTC-45-Prices.csv"     # ملف الأسعار + المؤشرات


#---------------------- Load strategy file ----------------------#

# توقع أسماء الأعمدة زي المثال اللي بعتهالي:
# Trade #	Type	Date/Time	Signal	Price USDT	Position size (qty)	Position size (value)
# Net P&L USDT	Net P&L %	Run-up USDT	Run-up %	Drawdown USDT	Drawdown %
# Cumulative P&L USDT	Cumulative P&L %

raw_trades = pd.read_csv(strategy_csv_path)

print("Strategy file columns:")
print(list(raw_trades.columns))

rename_map = {
    "Trade #": "trade_id",
    "Type": "type",
    "Date/Time": "datetime",
    "Signal": "signal",
    "Price USDT": "price",
    "Position size (qty)": "pos_qty",
    "Position size (value)": "pos_value",
    "Net P&L USDT": "net_pnl_usdt",
    "Net P&L %": "net_pnl_pct",
    "Run-up USDT": "runup_usdt",
    "Run-up %": "runup_pct",
    "Drawdown USDT": "drawdown_usdt",
    "Drawdown %": "drawdown_pct",
    "Cumulative P&L USDT": "cum_pnl_usdt",
    "Cumulative P&L %": "cum_pnl_pct",
}

# نعمل rename على الموجود بس
rename_map_existing = {k: v for k, v in rename_map.items() if k in raw_trades.columns}
raw_trades = raw_trades.rename(columns=rename_map_existing)

# تأكد إن عندنا الأعمدة المهمة
required_cols = ["trade_id", "type", "datetime", "price",
                 "net_pnl_pct", "runup_pct", "drawdown_pct"]
missing = [c for c in required_cols if c not in raw_trades.columns]
if missing:
    print("⚠️ Missing in strategy file:", missing)

# parse datetime
raw_trades["datetime"] = pd.to_datetime(raw_trades["datetime"])

# نرتب حسب رقم التريد و الوقت
raw_trades = raw_trades.sort_values(["trade_id", "datetime"]).reset_index(drop=True)

print("\nSample from strategy file after rename:")
print(raw_trades.head(8))


#---------------------- Build one-row-per-trade table ----------------------#

def build_trades_table(df):
    """
    لكل trade_id:
      - نلاقي صف الدخول (Type يحتوي 'Entry')
      - نلاقي صف الخروج (Type يحتوي 'Exit') لكن نتجاهل الصفوف اللي Signal فيها 'Open'
        (لأنها بتمثل صفقة لسه مفتوحة).
    """
    trades = []
    for tid, g in df.groupby("trade_id"):
        g = g.sort_values("datetime")

        # صفوف الدخول
        entry_rows = g[g["type"].str.lower().str.contains("entry")]

        # صفوف الخروج الحقيقية بس (signal فيها close مش open)
        exit_rows = g[
            g["type"].str.lower().str.contains("exit")
            & ~g["signal"].str.lower().str.contains("open")
        ]

        if len(entry_rows) == 0:
            continue

        entry = entry_rows.iloc[-1]   # آخر entry
        exit_row = exit_rows.iloc[-1] if len(exit_rows) > 0 else None

        trades.append({
            "trade_id": tid,
            "entry_time": entry["datetime"],
            "entry_price": float(entry["price"]),
            "exit_time": exit_row["datetime"] if exit_row is not None else pd.NaT,
            "exit_price": float(exit_row["price"]) if exit_row is not None else np.nan,
            "net_pnl_pct": float(entry.get("net_pnl_pct", np.nan)),
            "runup_pct": float(entry.get("runup_pct", np.nan)),
            "drawdown_pct": float(entry.get("drawdown_pct", np.nan)),
        })

    trades = pd.DataFrame(trades).sort_values("entry_time").reset_index(drop=True)
    return trades



#---------------------- Load prices + indicators file ----------------------#

prices = pd.read_csv(prices_csv_path)

print("\nPrices/indicators file columns:")
print(list(prices.columns))

# نعمل rename للأعمدة المهمة حسب اللي بعتهالي
price_rename = {
    "time": "time",
    "open": "open",
    "high": "high",
    "low": "low",
    "close": "close",
    "VWMA": "vwma",
    "Volume": "volume",
    "RSI": "rsi",
    "RSI-based MA": "rsi_ma",
    "Regular Bullish": "reg_bull",
    "Regular Bullish Label": "reg_bull_label",
    "Regular Bearish": "reg_bear",
    "Regular Bearish Label": "reg_bear_label",
    "%K": "stoch_k",
    "%D": "stoch_d",
    "Histogram": "macd_hist",
    "MACD": "macd",
    "Signal": "macd_signal",
    "OnBalanceVolume": "obv",
    "ATR": "atr",
    "ROC": "roc",
    "MOM": "mom",
    "CCI": "cci",
    "CCI-based MA": "cci_ma",
    "Conversion Line": "conv_line",
    "Base Line": "base_line",
    "Leading Span A": "span_a",
    "Leading Span B": "span_b",
}

price_rename_existing = {k: v for k, v in price_rename.items() if k in prices.columns}
prices = prices.rename(columns=price_rename_existing)

# parse time
if "time" not in prices.columns:
    raise ValueError("Prices file must contain a 'time' column.")
prices["time"] = pd.to_datetime(prices["time"])

# نرتب
prices = prices.sort_values("time").reset_index(drop=True)

print("\nSample from prices/indicators after rename:")
print(prices.head(10))


Strategy file columns:
['Trade #', 'Type', 'Date/Time', 'Signal', 'Price USDT', 'Position size (qty)', 'Position size (value)', 'Net P&L USDT', 'Net P&L %', 'Run-up USDT', 'Run-up %', 'Drawdown USDT', 'Drawdown %', 'Cumulative P&L USDT', 'Cumulative P&L %']

Sample from strategy file after rename:
   trade_id        type            datetime                     signal  \
0         1  Entry long 2024-01-03 08:30:00                       Long   
1         1   Exit long 2024-01-04 08:30:00  Close entry(s) order Long   
2         2  Entry long 2024-01-05 13:00:00                       Long   
3         2   Exit long 2024-01-05 19:00:00  Close entry(s) order Long   
4         3  Entry long 2024-01-07 22:45:00                       Long   
5         3   Exit long 2024-01-09 11:30:00  Close entry(s) order Long   
6         4  Entry long 2024-01-12 04:00:00                       Long   
7         4   Exit long 2024-01-15 14:30:00  Close entry(s) order Long   

     price   pos_qty    pos_value 

In [10]:
#===============================================================================
# CELL 2: Join indicators at trade entry & build ML dataset
#===============================================================================

# نستخدم نفس الأسامي اللي طلعت من Cell 1:
# trades  = جدول الصفقات (من build_trades_table)
# prices  = الأسعار + المؤشرات

# نتأكد إن الاتنين متسوّرين بالوقت
trades_sorted = trades.sort_values("entry_time").copy()
prices_sorted = prices.sort_values("time").copy()

# 🟢 أهم خطوة: توحيد نوع الـ datetime (ونخليهم UTC عشان timezone -05:00 في الأسعار)
trades_sorted["entry_time"] = pd.to_datetime(trades_sorted["entry_time"], utc=True)
prices_sorted["time"]       = pd.to_datetime(prices_sorted["time"], utc=True)

# دلوقتي نقدر نعمل merge_asof عادي: ياخد آخر بار <= entry_time
merged = pd.merge_asof(
    trades_sorted.sort_values("entry_time"),
    prices_sorted.sort_values("time"),
    left_on="entry_time",
    right_on="time",
    direction="backward"
)

print("Merged trades + features sample:")
print(merged.head(10))

# نشتغل فقط على الصفقات المقفولة للتدريب
closed = merged[~merged["exit_time"].isna()].copy()
open_for_pred = merged[merged["exit_time"].isna()].copy()

print(f"\nClosed trades for training: {len(closed)}")
print(f"Open trades for prediction: {len(open_for_pred)}")
if len(open_for_pred) > 0:
    print(open_for_pred[["trade_id", "entry_time", "entry_price", "time", "close"]].tail())

#---------------------- Define features & targets ----------------------#

FEATURE_COLS = [
    "rsi",
    "rsi_ma",
    "stoch_k",
    "stoch_d",
    "macd",
    "macd_signal",
    "macd_hist",
    "vwma",
    "volume",
    "atr",
    "roc",
    "mom",
    "cci",
    "cci_ma",
    "conv_line",
    "base_line",
    "span_a",
    "span_b",
]

existing_features = [c for c in FEATURE_COLS if c in closed.columns]
missing_feat = [c for c in FEATURE_COLS if c not in closed.columns]
if missing_feat:
    print("⚠️ Missing feature columns in merged data:", missing_feat)
print("\nUsing feature columns:")
print(existing_features)

closed_feat = closed.dropna(subset=existing_features).copy()

if "runup_pct" not in closed_feat.columns or "drawdown_pct" not in closed_feat.columns:
    raise ValueError("runup_pct / drawdown_pct not found in trades data.")

#---------------------- Bucketing functions ----------------------#

def bucket_runup(x):
    if x < 2:
        return 0
    elif x < 5:
        return 1
    elif x < 10:
        return 2
    else:
        return 3

def bucket_drawdown(x):
    if x > -1:
        return 0
    elif x > -3:
        return 1
    else:
        return 2

closed_feat["runup_class"] = closed_feat["runup_pct"].astype(float).apply(bucket_runup)
closed_feat["drawdown_class"] = closed_feat["drawdown_pct"].astype(float).apply(bucket_drawdown)

print("\nRun-up class distribution:")
print(closed_feat["runup_class"].value_counts().sort_index())
print("\nDrawdown class distribution:")
print(closed_feat["drawdown_class"].value_counts().sort_index())

#---------------------- Build X, y ----------------------#

X = closed_feat[existing_features].astype(float).values
y_run = closed_feat["runup_class"].values
y_dd  = closed_feat["drawdown_class"].values

print("\nML dataset shape:", X.shape)

n = len(X)
split_idx = int(n * 0.7)
X_train, X_test = X[:split_idx], X[split_idx:]
y_run_train, y_run_test = y_run[:split_idx], y_run[split_idx:]
y_dd_train,  y_dd_test  = y_dd[:split_idx],  y_dd[split_idx:]

print(f"Train size: {len(X_train)}, Test size: {len(X_test)}")


Merged trades + features sample:
   trade_id                entry_time  entry_price           exit_time  \
0         1 2024-01-03 08:30:00+00:00      42478.0 2024-01-04 08:30:00   
1         2 2024-01-05 13:00:00+00:00      43603.6 2024-01-05 19:00:00   
2         3 2024-01-07 22:45:00+00:00      43477.7 2024-01-09 11:30:00   
3         4 2024-01-12 04:00:00+00:00      45862.2 2024-01-15 14:30:00   
4         5 2024-01-17 13:00:00+00:00      42375.5 2024-01-18 03:15:00   
5         6 2024-01-18 13:00:00+00:00      41813.9 2024-01-19 03:15:00   
6         7 2024-01-19 13:00:00+00:00      41239.2 2024-01-20 15:15:00   
7         8 2024-01-21 12:15:00+00:00      41645.0 2024-01-24 03:15:00   
8         9 2024-01-24 15:15:00+00:00      39924.7 2024-01-24 20:30:00   
9        10 2024-01-29 10:45:00+00:00      42082.6 2024-01-29 12:15:00   

   exit_price  net_pnl_pct  runup_pct  drawdown_pct                      time  \
0     43242.1         1.80       2.63         -1.31 2024-01-03 08:15:00

In [11]:
#===============================================================================
# CELL 3: Train RandomForest models & predict for open trade
#===============================================================================

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

run_model = None
dd_model  = None

#---------------------- Train Run-up model ----------------------#

if len(np.unique(y_run_train)) > 1:
    run_model = RandomForestClassifier(
        n_estimators=400,
        min_samples_leaf=5,
        class_weight="balanced_subsample",
        random_state=42,
        n_jobs=-1,
    )
    run_model.fit(X_train, y_run_train)
    y_run_pred = run_model.predict(X_test)
    print("=== RUN-UP CLASS MODEL (Test) ===")
    print(classification_report(y_run_test, y_run_pred, digits=3, zero_division=0))
else:
    print("⚠️ Not enough class diversity in run-up to train model.")

#---------------------- Train Drawdown model ----------------------#

if len(np.unique(y_dd_train)) > 1:
    dd_model = RandomForestClassifier(
        n_estimators=400,
        min_samples_leaf=5,
        class_weight="balanced_subsample",
        random_state=42,
        n_jobs=-1,
    )
    dd_model.fit(X_train, y_dd_train)
    y_dd_pred = dd_model.predict(X_test)
    print("\n=== DRAWDOWN CLASS MODEL (Test) ===")
    print(classification_report(y_dd_test, y_dd_pred, digits=3, zero_division=0))
else:
    print("⚠️ Not enough class diversity in drawdown to train model.")


#---------------------- Mapping class -> human-readable text ----------------------#

run_bucket_desc = {
    0: "< 2%",
    1: "2% – 5%",
    2: "5% – 10%",
    3: "> 10%",
}

dd_bucket_desc = {
    0: "> -1% (low drawdown)",
    1: "-3% إلى -1% (moderate)",
    2: "< -3% (high risk)",
}

def describe_probs(probs, desc_map, title):
    print(f"\n🔮 {title} probability ranges:")
    for cls, p in enumerate(probs):
        if cls in desc_map:
            print(f"  - {desc_map[cls]:25s}: {p*100:5.1f}%")


#---------------------- Predict for LAST OPEN TRADE ----------------------#

if len(open_for_pred) == 0:
    print("\n⚠️ No open trade found to predict.")
else:
    # ناخد آخر صفقة مفتوحة (أحدث واحدة)
    last_open = open_for_pred.sort_values("entry_time").iloc[-1]
    print("\n============================================")
    print("📌 LAST OPEN TRADE (for prediction)")
    print("============================================")
    print(f"Trade ID   : {last_open['trade_id']}")
    print(f"Entry time : {last_open['entry_time']}")
    print(f"Entry price: {last_open['entry_price']:.2f}")
    if not pd.isna(last_open.get("net_pnl_pct", np.nan)):
        print(f"Current P&L: {float(last_open['net_pnl_pct']):+.2f}% (floating)")

    # نجهّز فيتشرز
    x_last = last_open[existing_features].astype(float).values.reshape(1, -1)

    if run_model is not None:
        run_probs = run_model.predict_proba(x_last)[0]
        describe_probs(run_probs, run_bucket_desc, "Expected MAX PROFIT (Run-up %)")

    else:
        print("\n⚠️ Run-up model not trained.")

    if dd_model is not None:
        dd_probs = dd_model.predict_proba(x_last)[0]
        describe_probs(dd_probs, dd_bucket_desc, "Expected MAX DRAWDOWN (Drawdown %)")

    else:
        print("\n⚠️ Drawdown model not trained.")


=== RUN-UP CLASS MODEL (Test) ===
              precision    recall  f1-score   support

           0      0.823     0.944     0.879        54
           1      0.500     0.083     0.143        12
           2      0.000     0.000     0.000         0

    accuracy                          0.788        66
   macro avg      0.441     0.343     0.341        66
weighted avg      0.764     0.788     0.745        66


=== DRAWDOWN CLASS MODEL (Test) ===
              precision    recall  f1-score   support

           0      0.692     0.231     0.346        39
           1      0.239     0.579     0.338        19
           2      0.000     0.000     0.000         8

    accuracy                          0.303        66
   macro avg      0.310     0.270     0.228        66
weighted avg      0.478     0.303     0.302        66


📌 LAST OPEN TRADE (for prediction)
Trade ID   : 220
Entry time : 2025-11-11 09:15:00+00:00
Entry price: 104340.00
Current P&L: -9.02% (floating)

🔮 Expected MAX PROFI